# Test for NeRmo2
This notebook contains necessary tests to help users getting familiar with NeRmo faster. 

The model is written in xml and it will be handled here with MuJoCo and MJX. 

The following things will be contained: 
- Joints
- Actuators
- Sensors
- Other special structures

## Revisions for the model
These revisions are necessary for the model to be implemented on MJX, since MJX does not handle some specific collisions, e.g., cylinders and boxes. 

- Remove `<flag override="enable"/>` in `basic_scene.xml`
- Set `m1_tail` geom from `cylinder` to `capsule` in `tail.xml`
- Four foot sensors `cylinder` -> `capsule` in `leg_xx_assets/xx_asset_exp.xml` (`xx=fl,fr,rl,rr`)
  - Format of the sensors: `foot_s_xx` (with `xx` as the placeholder)
- Cancel the contact by setting `contype` to 0 by adjusting `spec` of the model for:
  - `m_ss`: the servo for the spine
  - `servo_check_xx`
  - `thigh_up_link_1_xx`
  - `thigh_up_link_2_xx`
    - (`xx=fl,fr,rl,rr`)
- Enable full tail functions
  - `tail_assets/tail.xml` -> `tail_assets/tail_new.xml`
  - `tail_assets/tail_tendon.xml` -> `tail_assets/tail_tendon_new.xml`

In [1]:
# import necessary modules

## modules for mujoco
import mujoco as mj
from mujoco import mjx

## computational modules
import jax
from jax import numpy as jp
import numpy as np

## modules for visualization
import matplotlib.pyplot as plt
import mediapy as media
import time

Warp DeprecationWarning: The namespace `warp.context` will soon be removed from the public API. It can still be accessed from `warp._src.context` but might be changed or removed without notice.
Warp DeprecationWarning: The symbol `warp.context.Module` will soon be removed from the public API. Use `warp.Module` instead.
Warp DeprecationWarning: The symbol `warp.context.get_module` will soon be removed from the public API. Use `warp.get_module` instead.
Warp DeprecationWarning: The namespace `warp.math` will soon be removed from the public API. It can still be accessed from `warp._src.math` but might be changed or removed without notice.


In [2]:
# basic constants
MODEL_PATH = "./dynamic_4l.xml"

Current knowledge: 
* Four foot sensors can be safely replaced by **capsule**s. By visualizing these sensors, they just coincide with the extremities of the four feet. 
  * `foot_s_fl, foot_s_fr, foot_s_rl, foot_s_rr`
  * done.
* The cylinder in `tail.xml` can be safely replaced by a **capsule**. 
  * done.
* The cylinder for `m_ss` can have its contype to be 0
* `servo_check_fl` as a cylinder will raise no collision. Thus their contype can all be set to 0. 

In [40]:
# or: load mjspec from file and adjust spec
# dir(mj.MjSpec) # use mj.MjSpec.from_file
spec = mj.MjSpec.from_file(MODEL_PATH)
# try to get the geom m_ss, which is located at servo spine
m_ss = spec.geom("m_ss")
# m_ss.rgba[3] = 0
# try to set contype of m_ss to 0: success; does not influence the control
m_ss.contype = 0

suffix_list = ["fl", "fr", "rl", "rr"]
for suf in suffix_list:
    serfo_check = spec.geom(f"servo_check_{suf}")
    serfo_check.contype = 0
    thigh_up_link_1 = spec.geom(f"thigh_up_link_1_{suf}")
    thigh_up_link_1.contype = 0
    thigh_up_link_2 = spec.geom(f"thigh_up_link_2_{suf}")
    thigh_up_link_2.contype = 0

Try to use the tail... success! 
- `tail.xml` -> `tail_new.xml`
- `tail_tendon.xml` -> `tail_tendon_new.xml`

In [41]:
# compile the model
mj_model = spec.compile()
mj_data = mj.MjData(mj_model)
# finally render the model
import mujoco.viewer
mj.viewer.launch(mj_model, mj_data)

In [42]:
# place them on GPU device using MJX
mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.put_data(mj_model, mj_data)

c:\Users\Leo\miniforge3\envs\mujoco-play\Lib\site-packages\mujoco\mjx\_src\mesh.py:141: UserWarning: Mesh "servo_link_l" has a coplanar face with more than 20 vertices. This may lead to performance issues and inaccuracies in collision detection. Consider decimating the mesh.
  warnings.warn(
c:\Users\Leo\miniforge3\envs\mujoco-play\Lib\site-packages\mujoco\mjx\_src\mesh.py:141: UserWarning: Mesh "hip_l" has a coplanar face with more than 20 vertices. This may lead to performance issues and inaccuracies in collision detection. Consider decimating the mesh.
  warnings.warn(
c:\Users\Leo\miniforge3\envs\mujoco-play\Lib\site-packages\mujoco\mjx\_src\mesh.py:141: UserWarning: Mesh "thigh_down_l" has a coplanar face with more than 20 vertices. This may lead to performance issues and inaccuracies in collision detection. Consider decimating the mesh.
  warnings.warn(
c:\Users\Leo\miniforge3\envs\mujoco-play\Lib\site-packages\mujoco\mjx\_src\mesh.py:141: UserWarning: Mesh "foot_l" has a coplana

In [43]:
print(mj_data.qpos, type(mj_data.qpos))
print(mjx_data.qpos, type(mjx_data.qpos), mjx_data.qpos.devices())

[ 2.68283853e-04  7.37217417e-03  2.64222460e-02  9.99958205e-01
 -8.66147238e-03 -1.30758401e-05  2.92700740e-03  2.74445470e-07
  1.62582359e-05 -9.56138447e-02  2.67437074e-01 -5.60655589e-01
  4.36222702e-01 -1.00895681e-01  2.65240436e-01 -5.61637453e-01
  4.36953158e-01 -3.53396004e-03 -2.57415647e-03 -1.64927975e-03
 -1.35021050e-03  1.35134387e-03 -8.19293994e-02  1.94340857e-01
 -4.04901301e-01  3.18740371e-01 -7.52793246e-02  2.00620793e-01
 -4.10492688e-01  3.22973417e-01  1.52600350e-05  1.36906241e-05
  1.21973628e-05  1.07912016e-05  9.47692765e-06  8.25086186e-06
  7.10737698e-06  6.05101630e-06  5.07892140e-06  4.19154805e-06
  3.38770419e-06  2.67130334e-06  2.04002645e-06  1.49355068e-06
  1.03187812e-06  6.54723432e-07  3.61765326e-07  1.52759679e-07
  2.75258124e-08 -3.32999943e-02] <class 'numpy.ndarray'>
[ 2.6828385e-04  7.3721739e-03  2.6422245e-02  9.9995822e-01
 -8.6614722e-03 -1.3075840e-05  2.9270074e-03  2.7444548e-07
  1.6258236e-05 -9.5613845e-02  2.674370

## Joints
This sections lists all important joints of the model. 

List the joints for sensors and actuators first. 

List the following attributes of joints for quick reference: 
- `ref`
- `damping`
- `armature`

## Sensors

This section lists all sensors of the model. All information is concluded from the xml files. 

Since the types of sensors in MuJoCo are rich, one can feel free to add new sensors for any desired information. 

List the following types of sensors for quick reference. 
- `jointpos`: measures the angle of the joint (returns a scalar) (copied from `mjData.sensordata`)
- `touch`: measures the contact force (returns a scalar)
- `framepos`
- `framequat`
- `framelinvel`
- `accelerometer`
- `gyro`

In [ ]:
# <include file="leg_fl_assets/fl_sensor_actuator.xml"/>
## <jointpos name="thigh_joint_fl" joint="thigh_joint_fl"/>

## <jointpos name="leg_joint_fl" joint="leg_joint_fl"/>

## Actuators

This section lists all actuators of the model. 